In [1]:
import torch
import torchvision
import matplotlib 
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.empty_cache() 
# del model
# del trainloader
# del testloader
print(torch.cuda.memory_summary())


D:\Anaconda\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\Anaconda\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomPerspective(p=0.5),
     transforms.RandomRotation(degrees=(90, -90), fill=(0,))
     transforms.ColorJitter(brightness=(0.5,1.5),contrast=(1),saturation=(0.5,1.5),hue=(-0.1,0.1)),
     transforms.Resize((150,150)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.ImageFolder(root='Geo_data/seg_train', transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.ImageFolder(root='Geo_data/seg_test', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)



In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available(), torch.cuda.current_device(), torch.cuda.device_count(), torch.cuda.get_device_name(0))

True 0 1 NVIDIA GeForce GTX 1060 6GB


In [ ]:
class block(nn.Module):
    def __init__(
        self, in_channels, intermediate_channels, identity_downsample=None, stride=1
    ):
        super(block, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(
            in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False
        )
        self.bn1 = nn.BatchNorm2d(intermediate_channels)
        self.conv2 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(intermediate_channels)
        self.conv3 = nn.Conv2d(
            intermediate_channels,
            intermediate_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(intermediate_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [16]:
import torch.nn as nn
import torch.nn.functional as F

def calculate_accuracy():
    
    correct_images = 0
    total_images = 0
    with torch.no_grad(): 
        for images,labels in testloader:
            images, labels = images.to('cuda'),labels.to('cuda')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_images += labels.size(0)
            correct_images += (predicted == labels).sum().item()
        
        acc = 100 * correct_images // total_images
        return acc

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=6, padding=1)
        self.conv6 = nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=6, padding=1)

        self.res1 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(64),
                                  nn.ReLU(),
                                  nn.Conv2d(64, 64, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(64))
        
        self.res2 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(128),
                                  nn.ReLU(),
                                  nn.Conv2d(128, 128, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(128))

        self.res3 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(256),
                                  nn.ReLU(),
                                  nn.Conv2d(256, 256, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(256))
        
        self.res4 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(512),
                                  nn.ReLU(),
                                  nn.Conv2d(512, 512, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(512))
        
        self.res5= nn.Sequential(nn.Conv2d(1024, 1024, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(1024),
                                  nn.ReLU(),
                                  nn.Conv2d(1024, 1024, kernel_size=3, padding = 1),
                                  nn.BatchNorm2d(1024))
        
        
        self.batchnorm2d_1 = nn.BatchNorm2d(64)
        self.batchnorm2d_2 = nn.BatchNorm2d(128)
        self.batchnorm2d_3 = nn.BatchNorm2d(256)
        self.batchnorm2d_4 = nn.BatchNorm2d(512)
        self.batchnorm2d_5 = nn.BatchNorm2d(1024)
        self.batchnorm2d_6 = nn.BatchNorm2d(2048)

        self.pool = nn.MaxPool2d(2, 2)
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Dropout(0.2),
                                        nn.Linear(8192, 120),
                                        nn.ReLU(),
                                        nn.Linear(120, 6))
    def forward(self, xb):
        x = F.relu(self.conv1(xb))
        x = self.batchnorm2d_1(x)

        x = F.relu(self.res1(x) + x)
        x = F.relu(self.res1(x) + x)
   
        x = F.relu(self.conv2(x))
        x = self.batchnorm2d_2(x)
        x = self.pool(x)

        x = F.relu(self.res2(x) + x)
        x = F.relu(self.res2(x) + x)
        
        x = F.relu(self.conv3(x))
        x = self.batchnorm2d_3(x)
        
        x = F.relu(self.res3(x) + x)
        x = F.relu(self.res3(x) + x)

        x = F.relu(self.conv4(x))
        x = self.batchnorm2d_4(x)
        x = self.pool(x)

        x = F.relu(self.res4(x) + x)
        x = F.relu(self.res4(x) + x)

        x = F.relu(self.conv5(x))
        x = self.batchnorm2d_5(x)
        x = self.pool(x)
      
        x = F.relu(self.res5(x) + x)
        x = F.relu(self.res5(x) + x)

        x = F.relu(self.conv6(x))
        x = self.batchnorm2d_6(x)
        x = self.pool(x)
        x = self.classifier(x)

        return x
    
    def train_step(self, images, targets):
        images, targets = images.to(device), targets.to(device)
        predictions = self(images)
        loss = F.cross_entropy(predictions, targets)
        return loss

    def validate_batch(self, images, targets):
        images, targets = images.to(device), targets.to(device)
        preds = self(images)
        loss = F.cross_entropy(preds, targets)
        acc = self.accuracy(preds, targets)
        return {"loss": loss, "acc":acc}

    def evaluate(self):
        #generate output for every batch in the test data and evaluate the epoch

        output = [self.validate_batch(images.to(device), targets.to(device)) for images, targets in testloader]
        epoch_result = self.validate_epoch(output)

        return epoch_result

    def accuracy(self, outputs, targets):
        _, preds = torch.max(outputs, dim=1)
        acc = torch.tensor(torch.sum(preds == targets).item() / len(preds))
        return acc

    def validate_epoch(self, outputs):

        batch_accs = [x["acc"] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()

        loss = [x["loss"] for x in outputs]
        epoch_loss = torch.stack(loss).mean()

        return {"acc": epoch_acc, "loss": epoch_loss}

model = CNN()
model = model.to(device)

In [18]:
import torch.optim as optim

max_lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=max_lr, weight_decay=1e-4, momentum = 0.9)

sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=100, 
                                                steps_per_epoch=len(trainloader))

## Train the network

In [19]:
from datetime import datetime


print("Training on ", torch.cuda.get_device_name(0))
epochs = 100
for epoch in range(epochs):

    running_loss = 0.0

    for i,batch in enumerate(trainloader):
        
        images, targets = batch
        loss = model.train_step(images, targets)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        nn.utils.clip_grad_value_(model.parameters(), 0.1)
        
        running_loss += loss.item()
        
    sched.step()
    acc = calculate_accuracy()
    now = datetime.now()

    current_time = now.strftime("%H:%M:%S")
    print(f"Epoch [{epoch}]: loss: {running_loss/len(trainloader)}, accuracy: {acc}%, time finished: {now}")
    running_loss = 0.0


Training on  NVIDIA GeForce GTX 1060 6GB
Epoch [0]: loss: 1.2871226465414989, accuracy: 41%, time finished: 2022-04-08 13:20:23.558706
Epoch [1]: loss: 0.9247674599001129, accuracy: 49%, time finished: 2022-04-08 13:48:16.514686
Epoch [2]: loss: 0.7454270050736465, accuracy: 53%, time finished: 2022-04-08 14:16:04.880674
Epoch [3]: loss: 0.6486629911739009, accuracy: 60%, time finished: 2022-04-08 14:43:20.631758
Epoch [4]: loss: 0.5723570122575427, accuracy: 64%, time finished: 2022-04-08 15:10:54.699788
Epoch [5]: loss: 0.5186041552898171, accuracy: 66%, time finished: 2022-04-08 15:38:35.073646
Epoch [6]: loss: 0.48078528140377214, accuracy: 70%, time finished: 2022-04-08 16:07:48.163198
Epoch [7]: loss: 0.451666129149452, accuracy: 71%, time finished: 2022-04-08 16:35:35.273995
Epoch [8]: loss: 0.4165121330411844, accuracy: 73%, time finished: 2022-04-08 17:02:40.153434
Epoch [9]: loss: 0.39189946768292083, accuracy: 74%, time finished: 2022-04-08 17:29:42.351420
Epoch [10]: loss: 

KeyboardInterrupt: 

## Calculate layer output size 
Output of a conv layer : W-F+ 2xPadding/ stride + 1 <br>W= Input volume e.g 128x128 = 128 <br> F = Filter size

In [ ]:
width = 150
height = 150
stride = 3
padding = 3
filter_size = 3
number_of_filters = 32
input_channels = 3 #RGB

x = torch.zeros([1,input_channels,width,height])
conv1 = nn.Conv2d(in_channels = input_channels, out_channels = number_of_filters, kernel_size = filter_size, padding = padding, stride = stride)

#x = conv1(x)
print(f"Output width formular = {width} - {filter_size} + 2* {padding} / {stride} + 1 =", int((width-filter_size + 2 * padding)/stride + 1 ))
print(f"Output height formular = {height} - {filter_size} + 2* {padding} / {stride} + 1 =", int((height-filter_size + 2 * padding)/stride + 1 ))

print("Conv output shape: ", x.shape)


In [ ]:
x = torch.zeros([1,input_channels,width,height])

network = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding = 1),
            nn.Conv2d(32, 32, 5, padding = 1)
            )

print(x.shape)
print(network(x).shape)

network = nn.Sequential(
            nn.Conv2d(3, 32, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
    
            nn.Conv2d(32, 64, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
    
            nn.Flatten(),
            nn.Linear(73984, 150),
            nn.ReLU(),
            nn.Linear(150, 84),
            nn.ReLU(),
            nn.Linear(84, 6))



In [ ]:
print(len(trainloader))

In [ ]:
print(f'Accuracy of the network: {100 * correct_images // total_images} %')


In [20]:
torch.save(model.state_dict(), "model_83")

In [21]:
print(model)

CNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(1024, 2048, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (res1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, af